# Make prediction with trained DeepSDM

## Load packages

In [1]:
import time
import torch
import pytorch_lightning as pl
from types import SimpleNamespace
import mlflow
from pytorch_lightning.strategies import DDPStrategy
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from LitDeepSDMData import LitDeepSDMData
from LitUNetSDM import LitUNetSDM
import os
from matplotlib import pyplot as plt
import yaml

## Assign the experiment_id and run_id of mlflow of which model will be predicted

In [2]:
experiment_id = '296382531977234002'
run_id = '2f07dfdd1bef43988e07f22cb2e322c5'
logged_path = f'mlruns/{experiment_id}/{run_id}'

## Model configuration
Some settings do not matter here. Just copy these configs from 02_train_deepsdm.py

In [3]:
# load configurations
yaml_conf_logged = f'{logged_path}/artifacts/conf/DeepSDM_conf.yaml'
with open(yaml_conf_logged, 'r') as f:
    DeepSDM_conf = yaml.load(f, Loader = yaml.FullLoader)
DeepSDM_conf = SimpleNamespace(**DeepSDM_conf)

## Load parameters from the checkpoint of a trained model

Check which top-k models are logged

In [5]:
checkpoint_path = f'{logged_path}/checkpoints'
checkpoints = [f for f in os.listdir(checkpoint_path) if f.endswith('.ckpt') and f != 'last.ckpt']
checkpoints

['epoch=26-step=3132.ckpt',
 'epoch=20-step=2436.ckpt',
 'epoch=27-step=3248.ckpt']

 - Read the logged average state_dict of top-k models
 - In function `LitUNetSDM`, set `predict_attention = True` to write the attention score map (default be False)

In [6]:
avg_state_dict = torch.load(f'{logged_path}/artifacts/top_k_avg_state_dict/top_k_avg_state_dict.pt')
model = LitUNetSDM(yaml_conf_logged, predict_attention = False).cuda()
model.load_state_dict(avg_state_dict)

<All keys matched successfully>

## Initialize datamodel

In [7]:
deep_sdm_data = LitDeepSDMData(yaml_conf_logged)

./workspace/cooccurrence_vector.json
./workspace/env_information.json
./workspace/k_information.json
./workspace/species_information.json


## Select the species and dates for prediction.
 - The format of date must be YYYY-MM-01
 - If `species_list` or `date_list` is too big to load, sperate it and run this entire .ipynb multiple times

In [8]:
# The Carpodacus_formosanus is a species unknown to the model in the training process.
# We can still predict its distribution with help of species embeddings.
# We can also assign multiple species and dates for batch predictions. 
predict_dataloaders = deep_sdm_data.predict_dataloader(
    species_list = DeepSDM_conf.training_conf['species_list_predict'][60:65], 
    date_list = DeepSDM_conf.training_conf['date_list_train'])
# predict_dataloaders = deep_sdm_data.predict_dataloader(
#     species_list = DeepSDM_conf.training_conf['species_list_resident'], 
#     date_list = ['2018-01-01', '2018-07-01', '2018-10-01'])
# predict_dataloaders = deep_sdm_data.predict_dataloader(
#     species_list = DeepSDM_conf.training_conf['species_list_predict'], 
#     date_list = DeepSDM_conf.training_conf['date_list_predict'])

Setting up dataset for prediction...


## Start prediction
The results inlcuding png images and geotiff will be output to the `output_dir`

In [ ]:
model.eval()
with torch.no_grad():
    raw_results = model.predict(predict_dataloaders, datamodule = deep_sdm_data, output_dir = f'predicts/{run_id}')

## Plot prediction results

In [ ]:
raw_results.keys()

In [ ]:
plt.imshow(
    raw_results['Prinia_flaviventris_2018-09-01'][1],
    cmap='jet',
)
plt.colorbar()